Summary
=======

In this notebook I want to derive the algorithm for constructing a PPM (*Prediction by Partial Match*) model for text content. This is based on

*"On Prediction Using Variable Order Markov Models"* by Ron Begleiter, Ran El-Yaniv, and Golan Yona in
the Journal of Artificial Intelligence Research 22 (2004) 385-421.

I have already implemented the code for this on github [here](https://github.com/rpgomez/vomm) but that was for the scenario in which the size of the alphabet is relative small ($\lessapprox 256$), not the case where the size of the alphabet is large ($\gtrapprox 2^{15}$) such as the vocabulary of a large collection of documents. 

In the second case, the amount of memory required to build a PPM model is significantly higher than necessary with a simple, naive approach. So I'm going to document here my approach for conserving on memory in order to implement a reasonable PPM model where the vocabulary are words and a sequence of symbols are a sequence of grammatically correct words.


## Estimating the probability distribution for a categorical variable $X$ with sparse data.

* I have a catecorical variable $X$ which takes integer values $1,2, \ldots, N$ whenre $N \gg 1$
* I have a set of i.i.d. drawn samples of $X$, $s_1,s_2,\ldots, s_M$ where $M \ll N$
* I want to estimate the probability distribution $Pr(X=i)$ from my samples.

What is the appropriate algorithm?

### Solution: Simplify the problem

I have a random variable $Y$ with 2 possible values $0,1$ with observed counts of $n_0, n_1$. The maximum likelihood estimator (MLE) for $Pr(Y=i)$ is given by 

$$Pr(Y=i) = \frac{n_i}{n_0 + n_1}$$

To estimate the uncertainty of my estimate I can use Bayesian inference with a [Dirichlet](https://en.wikipedia.org/wiki/Dirichlet_distribution) [Conjugate Prior](https://en.wikipedia.org/wiki/Conjugate_prior) to estimate $Pr(Y=i)$. I start by using the uninformative prior $Dir(\alpha= (1,1))$ and then I use the observed counts to update my belief to a posterior distribution:
$Dir(\alpha = (1 + n_0,1 + n_1))$. 

Using my posterior distribution I can now determine the most likely maximum a posteriori (MAP) distribution (the same as MLE in this case), but can also measure the variance of the possible distributions that explain the observed data.

### The more complicated problem
I have a categorical value $X$ with $N \gg 1$, but I only I get to make at most $M \ll N$ observations. How can I use the simplified problem to estimate $Pr(X=i)$?

**Solution**:

1. Without loss of generality, let $1,\ldots,R$ be the observed values in the sequence $s_1,\ldots, s_M$.
2. Let $Y$ denote a random variable which takes on values in the range $1,\ldots, R$.
3. Let $Z$ denote a random variable which takes on values in the range $R+1,\ldots, N$.
4. We can interpret $X$ a mixture of the 2 random variables $Y,Z$ as follows:
\begin{align}
Pr(X = i) &= Pr(X=i, Y \text{ is selected}) + Pr(X=i, Z\text{ is selected}) \\
&= Pr(X=i| Y \text{ is selected}) Pr(Y \text{ is selected}) + Pr(X=i| Z\text{ is selected})Pr( Z\text{ is selected}) \\
&= Pr(X=i| Y \text{ is selected}) \omega + Pr(X=i| Z\text{ is selected})(1-\omega) \\
&= Pr(Y=i| Y \text{ is selected}) \omega + Pr(Z=i| Z\text{ is selected})(1-\omega) \\
&= Pr(Y=i) \omega + Pr(Z=i)(1-\omega) \\
\end{align}
where $0 \le \omega \le 1$ is the mixing parameter.

We can estimate $\omega$ using the simplified problem:

$$(\omega, 1-\omega) \sim Dir\left(\alpha = (1 + n_Y, 1 + n_Z)\right) = Dir\left(\alpha = (1 + M, 1)\right)$$

whose mean value is given $ \omega = (1 + M)/(2 + M)$. We should not use the MAP/MLE value:
$\omega = M/M = 1$ as that implies observations of $Z$ never occur.



We can estimate the probability distribution for $Y$ using the simplified problem.

Let $c_i, i = 1, \ldots, R$ be the observed frequency counts of $X=i$. Then $$Y \sim Dir(\alpha = ( 1+c_1, \ldots, 1 + c_R))$$

We can choose to use the mean value distribution for $Y$:

$$Pr(Y=i) = \frac{1 + c_i}{R + \sum_j c_j}$$

or the MLE/MAP:

$$Pr(Y=i) = \frac{c_i}{\sum_j c_j}$$


Because we never get to observe $Z$ we only have the uninformative prior $Z \sim Dir(\alpha = (1,1,\ldots,1))$ whose mean distribution for $Z$ is $Pr(Z = i) = 1/(N - R)$. Because $1-\omega \ll \omega$ using the mean distribution should not cause significant errors in predictions.

### Estimating $Pr(word| context)$

From above we can now estimate for a given observed context of size $d$ the probability distribution of the word that follows. 

In order to conserve memory 

* we first translate all words into numbers, $1, 2 , \ldots, W$ where $W$ is the size of the vocabulary
* create a dictionary *Counts* with keys all observed contexts of size $d$ and values another dictionary where
    * the value is a dictionary that has (key, value) pairs (word, # of occurrences after the specific context), i.e.
     Counts(c)(w) = # of times word *w* was observed to follow context *c*
* create a dictionary *Probabilities* 
    * with keys all observed contexts of size $d$ 
    * and for each key a dictionary with (w, p) pair where 
        * *w* is a word observed to follow the context 
        * and p is the probability of *w* occurring given by 
        $$\omega_{c}\times c_w/\sum_v c_v$$ where 
            * $\omega_c$ is the mixing factor determined for the given context $c$, 
            * and $c_v$ is the number of times word $v$ was observed to follow context $c$.
        * There is 1 additional (w,p) pair with key  *none* and value $(1-\omega_c)/(W-R_c)$ where
            * $W-R_c$ is the number of vocabulary words that were **not** observed to follow context $c$.
        

     



In [ ]:
%pylab inline

In [ ]:
from collections import Counter
from collections import defaultdict

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def construct_counts_dictionary(observed_sequence,d=4,debug=False):
    """Constructs the dictionary of observed counts of words following 
    contexts at most d words long. 
    
    observed_sequence should be a sequence of type integer of non-negative integers.
    Each integer corresponds to some word in the vocabulary.
    """
    
    if not debug:
        def use_me(x):
            return x
    else:
        use_me = tqdm
            
    counts = defaultdict(Counter)
    N = len(observed_sequence)
    
    for t in use_me(range(N)):
        obs = observed_sequence[t]
        for l in range(0,d+1):
            if t - l < 0:
                break
            context = observed_sequence[t-l:t]
            counts[context][obs] +=1
    
    return counts
            
def construct_prob_dictionary(counts_dictionary,vocabulary_size,debug=False):
    """Computes the memory efficient dictionary word| context probabilities"""

    V = vocabulary_size
    
    if not debug:
        def use_me(x):
            return x
    else:
        use_me = tqdm
            
    probs_dictionary = dict()
    for context in use_me(counts_dictionary):
        local_list = counts_dictionary[context]
        total_count = sum(list(local_list.values()))
        
        R = len(local_list)
        if R == V:
            omega = 1.0
        else:
            omega = (1+ total_count)/(2 + total_count)
            
        probs_dictionary[context] = defaultdict(float)
        
        local_probs = defaultdict(float)
        for word in local_list:
            c_i = local_list[word]
            pr = c_i/total_count * omega
            local_probs[word] = pr
        
        if R < V:
            local_probs[None] = (1.0 - omega)/(V-R)
        else:
            local_probs[None] = 0.0
        
        probs_dictionary[context] = local_probs
    
    return probs_dictionary

In [ ]:
def generate_log_pdf_dict(probs_dictionary,debug=False):
    """Computes the log likelihood dictionary
    log (Pr(symbol|context)) from the probs_dictionary"""
    
    if not debug:
        def use_me(x):
            return x
    else:
        use_me = tqdm
    
    log_pdf_dict = {}
    for context in use_me(probs_dictionary):
        local_pdf_list = probs_dictionary[context]
        local_log_pdf_list = {}
        for asymbol in local_pdf_list:
            local_log_pdf_list[asymbol] = np.log(local_pdf_list[asymbol])
            
        log_pdf_dict[context] = local_log_pdf_list
    
    return log_pdf_dict

In [ ]:
class PPM_words:
    """This class is designed to be memory efficient in the case that the size of the vocabulary is large
    (>= 2**15 for example)."""
    
    def __init__(self):
        """ Not much to do here. """

    def generate_fast_lookup(self,verbose=False):
        """Takes the pdf_dict dictionary and computes a faster lookup
        dictionary mapping suffix s to its longer contexts xs.  I need
        this to speed up computing the probability of logpdf for an
        observed sequence
        """

        if not verbose:
            def use_me(x,desc=None):
                return x
        else:
            use_me = tqdm

        # I want to create a fast look up of context s -> xs
        # So scoring a sequence is faster.
        
        context_lengths = [0]*(self.d+1)
        for context in self.pdf_dict:
            try:
                l = len(context)
            except:
                l  = 0
                
            context_lengths[l] += 1
        context_by_length  = dict([(k,[0]*context_lengths[k]) for k in range(self.d+1) ])

        indices = [0]*(self.d+1)
        for x in use_me(self.pdf_dict.keys(),desc='populating context_by_length'):
            try:
                l = len(x)
            except:
                l = 0
            t = indices[l]
            context_by_length[l][t] = x
            indices[l] += 1
            

        # Now lets generate a dictionary look up context s -> possible context xs.
        self.context_child = {}

        for k in range(self.d):
            for x in use_me(context_by_length[k],desc='context_child %d' %k):
                self.context_child[x] = [ y for y in context_by_length[k+1] if y[1:] == x ]

        for x in context_by_length[self.d]:
            self.context_child[x] = []

    def fit(self,training_data, d=4, alphabet_size = None,verbose=False):
        """
        This is the method to call to fit the model to the data.
        training_data should be a sequence of symbols represented by
        integers 0 <= x < alphabet size.

        d specifies the largest context sequence we're willing to consider.

        alphabet_size specifies the number of distinct symbols that
        can be possibly observed. If not specified, the alphabet_size
        will be inferred from the training data.
        """

        if alphabet_size == None:
            alphabet_size = max(training_data) + 1

        self.alphabet_size = alphabet_size
        self.d = d

        counts = construct_counts_dictionary(training_data,d=d,debug=verbose)
        self.pdf_dict = construct_prob_dictionary(counts,alphabet_size,debug=verbose)

        self.logpdf_dict = generate_log_pdf_dict(self.pdf_dict,debug=verbose)

        # For faster look up  when computing logpdf(observed data).
        self.generate_fast_lookup(verbose=verbose)

        return

    def logpdf(self,observed_data):
        """Call this method after using fitting the model to compute the log of
        the probability of an observed sequence of data.

        observed_data should be a sequence of symbols represented by
        integers 0 <= x < alphabet_size. """

        temp = tuple(observed_data)
        # start with the null context and work my way forward.

        logprob = 0.0
        for t in range(len(temp)):
            chunk = temp[max(t-self.d,0):t]
            sigma = temp[t]
            context = find_largest_context(chunk,self.context_child,self.d)
            if sigma in self.logpdf_dict[context]:
                logprob += self.logpdf_dict[context][sigma]
            else:
                logprob += self.logpdf_dict[context][None] # Didn't see this symbol with the context
        return logprob

    def generate_data(self,prefix=None, length=200,verbose=False):
        """Generates data from the fitted model.

        The length parameter determines how many symbols to generate.

        prefix is an optional sequence of symbols to be appended to,
        in other words, the prefix sequence is treated as a set of
        symbols that were previously "generated" that are going to be
        appended by an additional "length" number of symbols.

        The default value of None indicates that no such prefix
        exists. We're going to be generating symbols starting from the
        null context.

        It returns the generated data as an array of symbols
        represented as integers 0 <=x < alphabet_size.

        """

        if not verbose:
            def use_me(x):
                return x
        else:
            use_me = tqdm

        if prefix != None:
            new_data = np.zeros(len(prefix) + length,dtype=np.int)
            new_data[:len(prefix)] = prefix
            start = len(prefix)
        else:
            new_data = np.zeros(length,dtype=np.int)
            start = 0

        scratch_pdf = np.zeros(self.alphabet_size)
        for t in use_me(range(start,len(new_data))):
            chunk = tuple(new_data[max(t-self.d,0):t])
            context = find_largest_context(chunk,self.context_child,self.d)
            
            scratch_pdf[:] = self.pdf_dict[context][None] # the symbols we didn't see
            for symbol in self.pdf_dict[context]:
                if symbol  == None:
                    continue
                scratch_pdf[symbol] = self.pdf_dict[context][symbol]
                
            new_symbol = np.random.choice(self.alphabet_size,p=scratch_pdf)
            new_data[t] = new_symbol

        return new_data[start:]

    def __str__(self):
        """ Implements a string representation to return the parameters of this model. """

        return "\n".join(["alphabet size: %d" % self.alphabet_size,
                          "context length d: %d" % self.d,
                          "Size of model: %d" % len(self.pdf_dict)])


# Moby Dick
I'm going to create a PPM model for the content of Moby Dick.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
import nltk.tokenize

In [ ]:
import string

In [ ]:
nltk.tokenize.word_tokenize?

In [ ]:
nltk.download('punkt')

In [ ]:
content = open('mobydick.txt','r').read().lower()

In [ ]:
filter = string.ascii_letters
def filter_nonletters(aletter):
 if aletter in filter:
    return aletter
 else:
    return " "
new_content = "".join([filter_nonletters(x) for x in tqdm(content)])

In [ ]:
words = nltk.tokenize.word_tokenize(new_content)

In [ ]:
len(words)

In [ ]:
vocabulary = set(words)
print("Size of vocabulary: ", len(vocabulary))

In [ ]:
numbers_to_words = list(vocabulary)
words_to_numbers = {}
for t,word in enumerate(numbers_to_words):
    words_to_numbers[word] = t

In [ ]:
obs_sequence = tuple([words_to_numbers[word] for word in words])

In [ ]:
obs_sequence[:10]

In [ ]:
my_model = PPM_words()

In [ ]:
my_model.fit(obs_sequence,verbose=True)

In [ ]:
words[-10:]